In [1]:
import sys
sys.path.append('../')
from omegaconf import OmegaConf, DictConfig
from steer.vector_generators.vector_generators import BaseVectorGenerator
from steer.datasets import prepare_train_dataset
from steer.vector_appliers.vector_applier import BaseVectorApplier
from steer.datasets import prepare_generation_datasets
# model_path="/home/ubuntu/models/Deepseek-R1-Distill-Qwen-7B"

top_cfg = OmegaConf.load("./config_r1_control.yaml")
# top_cfg.model_name_or_path = model_path
top_cfg.device = "cuda:0"
top_cfg

{'model_name_or_path': '/home/ubuntu/models/Deepseek-R1-Distill-Qwen-7B', 'torch_dtype': 'bf16', 'device': 'cuda:0', 'use_chat_template': True, 'system_prompt': '', 'steer_train_hparam_paths': ['../hparams/Steer/caa_hparams/generate_caa.yaml'], 'steer_train_dataset': ['r1'], 'steer_vector_output_dir': 'vectors/Deepseek-R1-Distill-Qwen-7B', 'apply_steer_hparam_paths': ['../hparams/Steer/caa_hparams/apply_caa.yaml'], 'steer_vector_load_dir': ['vectors/Deepseek-R1-Distill-Qwen-7B/r1/caa_vector'], 'generation_data': ['r1'], 'generation_data_size': 5, 'generate_orig_output': True, 'generation_output_dir': 'vectors/Deepseek-R1-Distill-Qwen-7B/r1/results2', 'num_responses': 1, 'steer_from_end_position': False, 'generation_params': {'max_new_tokens': 500, 'temperature': 1.0}}

### Generate Steering Vector

In [2]:
datasets = {
    'r1':[
        {'question': '1 + 1 = ', 
        'matching':'</think>\n\n1 + 1 equals 2. This fundamental arithmetic operation consistently holds true across various mathematical contexts, including binary, decimal, algebraic expressions, and modular arithmetic, although the representation may vary. In standard arithmetic, the sum of two ones is always two.<｜end▁of▁sentence｜>', 
        'not_matching': "Alright, so I'm trying to figure out what 1 + 1 equals. Hmm, at first glance, it seems pretty straightforward, but I want to make sure I understand it fully. Let me think about how addition works. When you add two numbers, you're combining their quantities. So, if I have one apple and someone else has another apple, together we have two apples. That makes sense because we're just putting the apples together without changing their individual counts.\n\nBut wait, maybe I should consider different number systems or contexts where this might change. For example, in binary, which is the base-2 system, 1 + 1 equals 10. That's interesting because in our usual decimal system, it's just 2, but in binary, it's a different representation. So, the way we add numbers can vary depending on the base we're using.\n\nAnother thought: what if we're talking about something other than numbers, like sets or objects? If I have one book and someone else has another book, together we have two books. It's the same concept, just adding the quantities. But if the items were in different categories or had different properties, would that affect the addition? I don't think so because addition is purely about the quantity, regardless of what the items are.\n\nI also wonder about the history of addition. How did humans figure out that combining two quantities gives a sum? It must have been through counting and recognizing patterns. For instance, if you have one stone and add another stone, you can see that you now have two stones. This simple concept likely formed the basis of mathematical addition.\n\nWhat about in mathematics, specifically in algebra? If I have variables, say x + x, that simplifies to 2x. So, in that case, 1 + 1 would be 2. It's consistent with the basic arithmetic we learned earlier. But what if it's more complex, like adding fractions or decimals? For example, 1/2 + 1/2 equals 1, and 0.5 + 0.5 also equals 1. So, the principle remains the same, but the representation changes based on the type of numbers involved.\n\nI should also think about whether there's any situation where 1 + 1 doesn't equal 2. In standard mathematics, across all number systems, 1 + 1 equals 2. Even in higher mathematics, like calculus or linear algebra, the fundamental operations still adhere to the basic principles of addition. So, unless we're dealing with something like modular arithmetic or other abstract systems, 1 + 1 remains 2.\n\nWait, in modular arithmetic, 1 + 1 modulo 2 would be 0. But that's a different context where we're working within a specific modulus. So, it's still 2 in the usual sense, but modulo 2, it's 0. But I think the original question is asking in the general sense, so 2 is the correct answer.\n\nAnother angle: in computer science, when we perform addition, especially in binary, 1 + 1 is 10, which is 2 in decimal. So, the result is the same, just represented differently. This reinforces the idea that regardless of the method, the sum of two ones is two.\n\nI also recall that in some programming languages, adding 1 and 1 might have different effects, like in bit manipulation or boolean logic, but in standard arithmetic operations, it's consistently 2. So, unless specified otherwise, 1 + 1 equals 2.\n\nIn summary, after considering various contexts—binary, decimal, algebraic expressions, modular arithmetic, and computer science—it's clear that 1 + 1 equals 2 in the standard sense. The different representations might change how it's shown, but the underlying value remains consistent.\n</think>\n\n1 + 1 equals 2. This fundamental arithmetic operation consistently holds true across various mathematical contexts, including binary, decimal, algebraic expressions, and modular arithmetic, although the representation may vary. In standard arithmetic, the sum of two ones is always two.<｜end▁of▁sentence｜>"
        }
    ]
}

# Or use the datasets from config.yaml
# datasets = prepare_train_dataset(top_cfg)

vector_generator = BaseVectorGenerator(top_cfg)
vector_generator.generate_vectors(datasets)

CAA Generator Hyperparameters:
CAAHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bf16', seed=42, model_name_or_path='/home/ubuntu/models/Deepseek-R1-Distill-Qwen-7B', device='cuda:0', use_cache=True, generate_orig_output=True, alg_name='caa', layers=[17], steer_train_dataset=['r1'], multiple_choice=False, steer_vector_output_dir='vectors/Deepseek-R1-Distill-Qwen-7B', save_vectors=True)
Generating caa vectors ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing prompts: 100%|██████████| 1/1 [00:01<00:00,  1.78s/it]


Saving vectors to vectors/Deepseek-R1-Distill-Qwen-7B/r1/caa_vector ...


{'r1': {'caa': {'layer_17': tensor([ 0.6641, -0.4844, -0.1660,  ...,  0.4883,  0.0586,  1.0078],
          dtype=torch.bfloat16)}}}

### Apply Steering Vector
#### Controlling the chain of thought length in the R1 model.

In [2]:
vector_applier = BaseVectorApplier(top_cfg)
for mu in [-2, -1, 0, 1, 2]:
    vector_applier.hparams_dict["caa"].multipliers = [mu]
    vector_applier.apply_vectors()

    # You can customize your own inputs
    # datasets={f'r1_multipliers{mu}':[{'input':"A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?"}]}
    datasets={f'r1_multipliers{mu}':[{'input':"1 + 1 = "}]}

    # Or use the datasets from config.yaml
    # datasets = prepare_generation_datasets(top_cfg)

    # Method 1: Use parameters from config.yaml
    vector_applier.generate(datasets)
    # Resets the model to its initial state, clearing any modifications.
    vector_applier.model.reset_all()

CAA Applier Hyperparameters:
ApplyCAAHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bf16', seed=42, model_name_or_path='/home/ubuntu/models/Deepseek-R1-Distill-Qwen-7B', device='cuda:0', use_cache=True, generate_orig_output=True, alg_name='caa', layers=[17], multipliers=[2.0], steer_vector_load_dir='vectors/Deepseek-R1-Distill-Qwen-7B/r1/caa_vector')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Apply CAA to model: /home/ubuntu/models/Deepseek-R1-Distill-Qwen-7B
Loaded steering vector path:  vectors/Deepseek-R1-Distill-Qwen-7B/r1/caa_vector/layer_17.pt
Applying caa vectors to model successfully !



Evaluating dataset r1_multipliers-2: 100%|██████████| 1/1 [00:13<00:00, 13.62s/it]


Saving results to vectors/Deepseek-R1-Distill-Qwen-7B/r1/results2/r1_multipliers-2_results.json

===== r1_multipliers-2 Results =====

----- Input -----
1 + 1 = 

----- Orig Output-----
["First, I identify the two numbers involved in the addition, which are both 1.\n\nNext, I recall the basic addition fact that 1 plus 1 equals 2.\n\nTherefore, the sum of 1 and 1 is 2.\n</think>\n\nSure! Let's solve the addition problem step by step.\n\n**Problem:**  \n\\(1 + 1 = \\)\n\n**Solution:**\n\n1. **Identify the numbers to add:**  \n   We have two numbers, both are **1**.\n\n2. **Add the numbers together:**  \n   \\(1 + 1 = 2\\)\n\n3. **Final Answer:**  \n   \\(1 + 1 = \\boxed{2}\\)\n\n**Answer:**  \n\\(\\boxed{2}\\)"]

----- Steered Output-----
["I need to add these two numbers, 1 and 1, which are pretty small, but I should make sure I'm doing it right. Maybe I should write them down on paper to visualize the addition. Wait, if I'm not a kid anymore, why would I use paper? But I might be confu

...

Saving results to vectors/Deepseek-R1-Distill-Qwen-7B/r1/results2/r1_multipliers0_results.json

===== r1_multipliers0 Results =====

----- Input -----
1 + 1 = 

----- Orig Output-----
["First, I recognize that the expression is a simple addition problem involving the numbers 1 and 1.\n\nNext, I perform the addition: 1 plus 1 equals 2.\n\nTherefore, the final answer is 2.\n</think>\n\nCertainly! Let's solve the simple addition problem step by step.\n\n**Problem:**\n\\[ 1 + 1 = \\boxed{2} \\]\n\n**Step-by-Step Solution:**\n\n1. **Identify the Numbers to Add:**\n   - We have two numbers to add: **1** and **1**.\n\n2. **Perform the Addition:**\n   - Add the two ones together.\n   \\[\n   1 + 1 = 2\n   \\]\n\n3. **Final Answer:**\n   - The sum of **1** and **1** is **2**.\n\n\\[\n\\boxed{2}\n\\]"]

----- Steered Output-----
["First, I recognize that the problem is a simple addition of two numbers: 1 and 1.\n\nNext, I'll add the two ones together.\n\nThe sum of 1 and 1 equals 2.\n\nTherefore

Evaluating dataset r1_multipliers1: 100%|██████████| 1/1 [00:03<00:00,  3.52s/it]


Saving results to vectors/Deepseek-R1-Distill-Qwen-7B/r1/results2/r1_multipliers1_results.json

===== r1_multipliers1 Results =====

----- Input -----
1 + 1 = 

----- Orig Output-----
["I see the problem is to calculate 1 plus 1.\n\nFirst, I recognize that adding two units together results in a total quantity.\n\nSo, 1 plus 1 equals 2.\n\nTherefore, the answer is 2.\n</think>\n\nSure! Let's solve the addition problem step by step.\n\n**Question:**  \n\\[ 1 + 1 = \\]\n\n**Solution:**  \nTo solve \\( 1 + 1 \\), we perform the addition:\n\n1. **Add the two numbers:**  \n   \\[ 1 + 1 = 2 \\]\n\n**Answer:**  \n\\[\n\\boxed{2}\n\\]"]

----- Steered Output-----
['First, I recognize that the expression to solve is \\( 1 + 1 \\).\n\nNext, I perform the addition of the two numbers.\n\nFinally, I arrive at the solution: \\( 2 \\).\n</think>\n\n**Solution:**\n\n\\[\n1 + 1 = \\boxed{2}\n\\]']

Apply CAA to model: /home/ubuntu/models/Deepseek-R1-Distill-Qwen-7B
Loaded steering vector path:  vectors/

Evaluating dataset r1_multipliers2: 100%|██████████| 1/1 [00:02<00:00,  2.86s/it]

Saving results to vectors/Deepseek-R1-Distill-Qwen-7B/r1/results2/r1_multipliers2_results.json

===== r1_multipliers2 Results =====

----- Input -----
1 + 1 = 

----- Orig Output-----
['First, I recognize that the expression I need to evaluate is 1 + 1.\n\nNext, I perform the addition of the two numbers.\n\nFinally, I arrive at the conclusion that the sum of 1 and 1 is 2.\n</think>\n\n**Solution:**\n\nWe are asked to calculate the sum of \\(1\\) and \\(1\\).\n\n\\[\n1 + 1 = 2\n\\]\n\n**Answer:** \\boxed{2}']

----- Steered Output-----
['First, I recognize that the expression \\(1 + 1\\) represents the addition of two unit quantities.\n\nBy applying the basic principles of arithmetic, the sum of 1 and 1 is equal to 2.\n</think>\n\n**Solution:**\n\n\\[\n1 + 1 = \\boxed{2}\n\\]']

